In [21]:
import pandas as pd
import folium

# Step 1: Load the data
data = pd.read_csv('/Users/mehakrafiq/Library/CloudStorage/OneDrive-NationalUniversityofSciences&Technology/Rapidev/SmartFile.csv')

# Step 2: Sort the data by timestamp
data.sort_values('TimeStamp', inplace=True)

# Step 3: Create a map centered around the first coordinate
m = folium.Map(location=[data.iloc[0]['Latitude'], data.iloc[0]['Longitude']], zoom_start=13)

# Step 4: Plot the trajectory as a line on the map
for i in range(1, len(data)):
    folium.PolyLine([(data.iloc[i-1]['Latitude'], data.iloc[i-1]['Longitude']), 
                     (data.iloc[i]['Latitude'], data.iloc[i]['Longitude'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath = 'walk_pattern_map.html'
m.save(map_filepath)

map_filepath


'walk_pattern_map.html'

In [22]:
from geopy.distance import geodesic

# Step 1: Handle missing values by removing rows with NaN values in Latitude and Longitude columns
data.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Step 2: Remove rows with same Latitude and Longitude as the previous row
data = data.loc[(data['Latitude'].shift() != data['Latitude']) | (data['Longitude'].shift() != data['Longitude'])]

# Step 3: Remove random jumps by setting a threshold for the maximum allowable distance between consecutive points (e.g., 100 meters)
max_distance_m = 100
for i in range(1, len(data)):
    coord1 = (data.iloc[i-1]['Latitude'], data.iloc[i-1]['Longitude'])
    coord2 = (data.iloc[i]['Latitude'], data.iloc[i]['Longitude'])
    if geodesic(coord1, coord2).meters > max_distance_m:
        data.iloc[i, data.columns.get_loc('Latitude')] = data.iloc[i-1]['Latitude']
        data.iloc[i, data.columns.get_loc('Longitude')] = data.iloc[i-1]['Longitude']

# Display the first few rows of the preprocessed data
data.head()


,Soldier_ID,TimeStamp,HeartRate,Latitude,Longitude,Accelerometer_X,Accelerometer_Y,Accelerometer_Z,Gyro_X,Gyro_Y,Gyro_Z,User_Activity
0,0,2023-09-08 16:23:07.200,-10,33.64616,72.99664,-1.01695,2.40000,9.21237,0.51646,-0.85545,0.00549,NaN
1,0,2023-09-08 16:23:07.779,-10,33.64617,72.99666,-1.07198,2.73739,8.96830,0.51204,-0.85745,0.00845,NaN
4,0,2023-09-08 16:23:10.922,-10,33.64616,72.99666,-0.06939,1.24666,9.56411,0.56297,-0.82643,0.00436,NaN
10,0,2023-09-08 16:23:16.897,-10,33.64615,72.99666,-2.98864,0.38285,9.97568,0.44209,-0.89592,-0.03627,NaN
13,0,2023-09-08 16:23:19.875,-10,33.64615,72.99665,1.78505,-1.78744,11.81815,-0.33094,0.94339,-0.00965,NaN


In [23]:
# Create a map centered around the first coordinate of the preprocessed data
m = folium.Map(location=[data.iloc[0]['Latitude'], data.iloc[0]['Longitude']], zoom_start=13)

# Plot the trajectory as a line on the map using the preprocessed data
for i in range(1, len(data)):
    folium.PolyLine([(data.iloc[i-1]['Latitude'], data.iloc[i-1]['Longitude']), 
                     (data.iloc[i]['Latitude'], data.iloc[i]['Longitude'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath = 'walk_pattern_map_preprocessed.html'
m.save(map_filepath)

map_filepath

'walk_pattern_map_preprocessed.html'

In [24]:
# Apply a moving average filter with a window size of 5 to the Latitude and Longitude columns to smooth the data
window_size = 5
data['Latitude_smooth'] = data['Latitude'].rolling(window=window_size, center=True).mean()
data['Longitude_smooth'] = data['Longitude'].rolling(window=window_size, center=True).mean()

# Remove the rows with NaN values generated due to smoothing
data.dropna(subset=['Latitude_smooth', 'Longitude_smooth'], inplace=True)

# Create a map centered around the first coordinate of the smoothed data
m = folium.Map(location=[data.iloc[0]['Latitude_smooth'], data.iloc[0]['Longitude_smooth']], zoom_start=13)

# Plot the trajectory as a line on the map using the smoothed data
for i in range(1, len(data)):
    folium.PolyLine([(data.iloc[i-1]['Latitude_smooth'], data.iloc[i-1]['Longitude_smooth']), 
                     (data.iloc[i]['Latitude_smooth'], data.iloc[i]['Longitude_smooth'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath_smooth = 'walk_pattern_map_smooth.html'
m.save(map_filepath_smooth)

map_filepath_smooth


'walk_pattern_map_smooth.html'

In [25]:
# Convert the TimeStamp column to datetime data type
data['TimeStamp'] = pd.to_datetime(data['TimeStamp'])

# Set the TimeStamp column as the index
data.set_index('TimeStamp', inplace=True)

# Resample the data to obtain a single location per minute by taking the mean of the Latitude and Longitude values within each minute
data_resampled = data[['Latitude', 'Longitude']].resample('1T').mean().dropna()

# Create a map centered around the first coordinate of the resampled data
m = folium.Map(location=[data_resampled.iloc[0]['Latitude'], data_resampled.iloc[0]['Longitude']], zoom_start=13)

# Plot the trajectory as a line on the map using the resampled data
for i in range(1, len(data_resampled)):
    folium.PolyLine([(data_resampled.iloc[i-1]['Latitude'], data_resampled.iloc[i-1]['Longitude']), 
                     (data_resampled.iloc[i]['Latitude'], data_resampled.iloc[i]['Longitude'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath_resampled = 'walk_pattern_map_resampled.html'
m.save(map_filepath_resampled)

map_filepath_resampled


'walk_pattern_map_resampled.html'

In [26]:
# Resample the data to obtain a single location every 2 minutes by taking the mean of the Latitude and Longitude values within each 2-minute interval
data_resampled_2min = data[['Latitude', 'Longitude']].resample('2T').mean().dropna()

# Create a map centered around the first coordinate of the 2-minute resampled data
m = folium.Map(location=[data_resampled_2min.iloc[0]['Latitude'], data_resampled_2min.iloc[0]['Longitude']], zoom_start=13)

# Plot the trajectory as a line on the map using the 2-minute resampled data
for i in range(1, len(data_resampled_2min)):
    folium.PolyLine([(data_resampled_2min.iloc[i-1]['Latitude'], data_resampled_2min.iloc[i-1]['Longitude']), 
                     (data_resampled_2min.iloc[i]['Latitude'], data_resampled_2min.iloc[i]['Longitude'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath_resampled_2min = 'walk_pattern_map_resampled_2min.html'
m.save(map_filepath_resampled_2min)

map_filepath_resampled_2min


'walk_pattern_map_resampled_2min.html'

In [27]:
# Resample the data to obtain a single location every 5 minutes by taking the mean of the Latitude and Longitude values within each 5-minute interval
data_resampled_5min = data[['Latitude', 'Longitude']].resample('5T').mean().dropna()

# Create a map centered around the first coordinate of the 5-minute resampled data
m = folium.Map(location=[data_resampled_5min.iloc[0]['Latitude'], data_resampled_5min.iloc[0]['Longitude']], zoom_start=13)

# Plot the trajectory as a line on the map using the 5-minute resampled data
for i in range(1, len(data_resampled_5min)):
    folium.PolyLine([(data_resampled_5min.iloc[i-1]['Latitude'], data_resampled_5min.iloc[i-1]['Longitude']), 
                     (data_resampled_5min.iloc[i]['Latitude'], data_resampled_5min.iloc[i]['Longitude'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath_resampled_5min = 'walk_pattern_map_resampled_5min.html'
m.save(map_filepath_resampled_5min)

map_filepath_resampled_5min


'walk_pattern_map_resampled_5min.html'

In [28]:
# Apply a moving average filter with a window size of 3 to the 2-minute resampled Latitude and Longitude columns to smooth the data
window_size = 3
data_resampled_2min['Latitude_smooth'] = data_resampled_2min['Latitude'].rolling(window=window_size, center=True).mean()
data_resampled_2min['Longitude_smooth'] = data_resampled_2min['Longitude'].rolling(window=window_size, center=True).mean()

# Remove the rows with NaN values generated due to smoothing
data_resampled_2min.dropna(subset=['Latitude_smooth', 'Longitude_smooth'], inplace=True)

# Create a map centered around the first coordinate of the smoothed and resampled data
m = folium.Map(location=[data_resampled_2min.iloc[0]['Latitude_smooth'], data_resampled_2min.iloc[0]['Longitude_smooth']], zoom_start=13)

# Plot the trajectory as a line on the map using the smoothed and resampled data
for i in range(1, len(data_resampled_2min)):
    folium.PolyLine([(data_resampled_2min.iloc[i-1]['Latitude_smooth'], data_resampled_2min.iloc[i-1]['Longitude_smooth']), 
                     (data_resampled_2min.iloc[i]['Latitude_smooth'], data_resampled_2min.iloc[i]['Longitude_smooth'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath_resampled_smooth_2min = 'walk_pattern_map_resampled_smooth_2min.html'
m.save(map_filepath_resampled_smooth_2min)

map_filepath_resampled_smooth_2min


'walk_pattern_map_resampled_smooth_2min.html'

In [29]:
# Apply a moving average filter with a window size of 3 to the original Latitude and Longitude columns to smooth the data
window_size = 5
data['Latitude_smooth'] = data['Latitude'].rolling(window=window_size, center=True).mean()
data['Longitude_smooth'] = data['Longitude'].rolling(window=window_size, center=True).mean()

# Remove the rows with NaN values generated due to smoothing
data.dropna(subset=['Latitude_smooth', 'Longitude_smooth'], inplace=True)

# Resample the smoothed data to obtain a single location every 1 minute
data_resampled_smooth_1min = data[['Latitude_smooth', 'Longitude_smooth']].resample('1T').mean().dropna()

# Create a map centered around the first coordinate of the 1-minute resampled and smoothed data
m = folium.Map(location=[data_resampled_smooth_1min.iloc[0]['Latitude_smooth'], data_resampled_smooth_1min.iloc[0]['Longitude_smooth']], zoom_start=13)

# Plot the trajectory as a line on the map using the 1-minute resampled and smoothed data
for i in range(1, len(data_resampled_smooth_1min)):
    folium.PolyLine([(data_resampled_smooth_1min.iloc[i-1]['Latitude_smooth'], data_resampled_smooth_1min.iloc[i-1]['Longitude_smooth']), 
                     (data_resampled_smooth_1min.iloc[i]['Latitude_smooth'], data_resampled_smooth_1min.iloc[i]['Longitude_smooth'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath_resampled_smooth_1min = 'walk_pattern_map_resampled_smooth_1min.html'
m.save(map_filepath_resampled_smooth_1min)

map_filepath_resampled_smooth_1min


'walk_pattern_map_resampled_smooth_1min.html'

In [30]:
# Apply a moving average filter with a window size of 3 to the Latitude and Longitude columns to smooth the data
window_size = 3
data['Latitude_smooth'] = data['Latitude'].rolling(window=window_size, center=True).mean()
data['Longitude_smooth'] = data['Longitude'].rolling(window=window_size, center=True).mean()

# Identify the unique Soldier_IDs in the data
unique_soldier_ids = data['Soldier_ID'].unique()

# Create a dictionary to store the maps for each Soldier_ID
maps_dict = {}

# Loop through each unique Soldier_ID and create a map for each
for soldier_id in unique_soldier_ids:
    # Filter the data for the current Soldier_ID
    data_filtered = data[data['Soldier_ID'] == soldier_id]
    
    # Resample the smoothed data to obtain a single location every 1 minute
    data_resampled_smooth_1min = data_filtered[['Latitude_smooth', 'Longitude_smooth']].resample('1T').mean().dropna()
    
    # Create a map centered around the first coordinate of the 1-minute resampled and smoothed data
    m = folium.Map(location=[data_resampled_smooth_1min.iloc[0]['Latitude_smooth'], data_resampled_smooth_1min.iloc[0]['Longitude_smooth']], zoom_start=13)
    
    # Plot the trajectory as a line on the map using the 1-minute resampled and smoothed data
    for i in range(1, len(data_resampled_smooth_1min)):
        folium.PolyLine([(data_resampled_smooth_1min.iloc[i-1]['Latitude_smooth'], data_resampled_smooth_1min.iloc[i-1]['Longitude_smooth']), 
                         (data_resampled_smooth_1min.iloc[i]['Latitude_smooth'], data_resampled_smooth_1min.iloc[i]['Longitude_smooth'])], color='blue').add_to(m)
        
    # Save the map to an HTML file
    map_filepath = f'/mnt/data/walk_pattern_map_resampled_smooth_1min_soldier_{soldier_id}.html'
    m.save(map_filepath)
    
    # Store the map filepath in the dictionary
    maps_dict[soldier_id] = map_filepath

maps_dict


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/walk_pattern_map_resampled_smooth_1min_soldier_0.html'

In [ ]:
# Drop the '0' rows from the 'soldier_id' column
#data = data[data['Soldier_ID'] != '0']
#data = data[data['Soldier_ID'] != 'Sarosh Sol']
#data.head()

,Soldier_ID,HeartRate,Latitude,Longitude,Accelerometer_X,Accelerometer_Y,Accelerometer_Z,Gyro_X,Gyro_Y,Gyro_Z,User_Activity,Latitude_smooth,Longitude_smooth
TimeStamp,,,,,,,,,,,,,
2023-09-13 11:22:05.571,Live Sol,91,33.64633,72.99696,6.48934,-0.59821,5.25225,-0.47323,0.79728,-0.29528,NaN,33.646313,72.997057
2023-09-13 11:22:06.465,Live Sol,92,33.64631,72.99697,9.14776,-1.06720,6.03470,-0.46169,0.77148,-0.32732,NaN,33.646280,72.997057
2023-09-13 11:22:10.464,Live Sol,98,33.64620,72.99724,7.80299,2.21336,5.57767,-0.49935,-0.75566,0.22874,NaN,33.646237,72.997193
2023-09-13 11:22:14.489,Live Sol,104,33.64620,72.99737,7.47039,-2.56511,5.85284,-0.46322,0.79019,-0.21714,NaN,33.646173,72.997360
2023-09-13 11:22:18.596,Live Sol,107,33.64612,72.99747,8.86062,2.87378,4.22572,-0.32873,-0.78414,0.27286,NaN,33.646147,72.997443


In [ ]:
# Apply a moving average filter with a window size of 3 to the original Latitude and Longitude columns to smooth the data
window_size = 3
data['Latitude_smooth'] = data['Latitude'].rolling(window=window_size, center=True).mean()
data['Longitude_smooth'] = data['Longitude'].rolling(window=window_size, center=True).mean()

# Remove the rows with NaN values generated due to smoothing
data.dropna(subset=['Latitude_smooth', 'Longitude_smooth'], inplace=True)

# Resample the smoothed data to obtain a single location every 1 minute
data_resampled_smooth_1min = data[['Latitude_smooth', 'Longitude_smooth']].resample('1T').mean().dropna()

# Create a map centered around the first coordinate of the 1-minute resampled and smoothed data
m = folium.Map(location=[data_resampled_smooth_1min.iloc[0]['Latitude_smooth'], data_resampled_smooth_1min.iloc[0]['Longitude_smooth']], zoom_start=20)

# Plot the trajectory as a line on the map using the 1-minute resampled and smoothed data
for i in range(1, len(data_resampled_smooth_1min)):
    folium.PolyLine([(data_resampled_smooth_1min.iloc[i-1]['Latitude_smooth'], data_resampled_smooth_1min.iloc[i-1]['Longitude_smooth']), 
                     (data_resampled_smooth_1min.iloc[i]['Latitude_smooth'], data_resampled_smooth_1min.iloc[i]['Longitude_smooth'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath_LiveSol_resampled_smooth_1min = 'walk_pattern_map_LiveSol_resampled_smooth_1min.html'
m.save(map_filepath_LiveSol_resampled_smooth_1min)

map_filepath_LiveSol_resampled_smooth_1min


'walk_pattern_map_LiveSol_resampled_smooth_1min.html'